## Intro to Neural Nets : Self-learning
### Abstract
The purpose of this notebook is to learn the basics of how neural networks work.
I will be *somewhat* following Packt's *Machine Learning with Pytorch and Scikit-learn,
starting with chapter 2's introduction to building simple discrete neurons, but will be 
integrating what I've previously learned from other sources. 

The resulting code is meant to be educational and illustrate concepts, not be 
performative; the final neural net won't be remotely optimized.  A later notebook will 
focus on implimenting a neural network in a more functional and practical manner using
numpy's optimized data structures and matrix functions.

*(additionally, I feel I need to review my linear algebra, so rather than leverage numpy 
I will instead be writing my own linear algebra functions in pure python)*

### Part 1: Building a neuron 
At it's most basic, an artificial neuron consists of three main components: weights; a bias; and an activation function.
- **weights:** Each neuron has one or more inputs. Each of these inputs is multiplied by a weight that determines how much and in what direction each of these inputs should affect the neuron's output. After the inputs have been weighed, they are then summed.
- **bias :** After the sum of weighed inputs has been computed, an additional value (+ or -) that is added to that sum.  This value makes it easier or harder for this neuron to turn on, effectively biassing it towards one state or another.
- **activation function:** Because the weights and biases are unbound, the sum of the biases and weighed inputs is also unbound in both directions -- this is unwanted.  The activation function is a final step that conditions the sum in such a manner that it is bound to be no less than 0 and possibly no more than 1 depending on the exact activation function used

First we need to be able to instantiate a neuron with those 3 basic components.  Let's write a constructor that initializes it with random weights and biases:

In [1]:
from typing import Callable
from random import random

class Neuron():

    def __init__(self, num_of_inputs: int, activation_function: str):
        """Instantiates a neuron with random weights and bias"""

        # validate args:
        if num_of_inputs < 1:
            print("there must be more than 1 input to a neuron")
            return

        self.num_of_inputs = num_of_inputs
        self.weights = [random() for i in range(num_of_inputs)]
        self.bias  = random()
        if activation_function == "RELU":
            self.activation_function = Neuron.RELU


# For an activation function, let's start with a simple RELU.
# Lets do this by extending the Neuron class with a class RELU method
# NOTE: Extending a class via self-inheritance isn't something I'd typically do!

class Neuron(Neuron):
    
    @classmethod
    def RELU(self,sum_of_weighed_inputs_and_bias):
        if sum_of_weighed_inputs_and_bias < 0:
            return(0)
        else: 
            return(sum_of_weighed_inputs_and_bias)


Now we need to add a method to allow this neuron to take a vector of inputs and compute an output.  This involves multiplying the transposed weights vector by the inputs vector and then adding the bias.  So first we need functions to transpose and multiply matrices:

In [2]:
def matrix_transpose(matrix):
    # If a matrix has n columns and m rows, its transpose
    # will have m columns and n rows, so let's build a 
    # zeros matrix with that shape:
    cols = len(matrix)
    rows = len(matrix[0])
    mT = [[0 for _ in range(cols)] for _ in range(rows)]
    
    # use generator comprehension to populate zeros matrix
    elements = (elem for row in matrix for elem in row)
        
    for col in range(cols):
        for row in range(rows):
            mT[row][col] = elements.__next__()

    return(mT)


def matrix_mult(A,B):
    #Check that inner dimensions are equal:
    assert len(A[0])==len(B)
    
    rows_A = len(A)
    len_n = len(B)
    cols_B = len(B[0])
    
    C = [[None for j in range(cols_B)] for i in range(rows_A)]
    
    for i in range(rows_A):
        for j in range(cols_B):
            A_row,B_col = A[i], [x[j] for x in B]
            C[i][j] = sum([A_row[k] * B_col[k] for k in range(len_n)])
        
    return(C)

Now that we have our matrix functions ready, let's define how a neuron calculates a preduction from inputs:

In [7]:
class Neuron(Neuron):

    def predict(self, inputs: list[float]):
        print(inputs,self.weights)
        # make sure we have exactly 1 weight for each input:
        assert(len(inputs)==len(self.weights))

        weighted_sum = matrix_mult( matrix_transpose([self.weights]) , [inputs]) 
        biased_weighted_sum = weighted_sum[0][0] + self.bias
        
        return(self.activation_function(biased_weighted_sum))

At this point, we have an implimentation of an artificial neuron that can be instantiated with 
random weights and used in computation.  All that's missing is the ability to train the neuron, but before we can think about how to train a neuron, we first need data to train against.  Let's assume we have 2 predictors, x1 and x2, and are trying to predict class Y whose members are Red and Blue

In [4]:
# create pairs of random numbers (x1,x2) between -10 and 10
data = [[(random() - .5) * 20, (random() - .5) * 20] for _ in range(10)]

# if x1 is greater, assign that data point to class 0, else class 1:
for data_point in data:
    if data_point[0] >  data_point[1]:
        data_point.append(0)
    else:
        data_point.append(1)

In [9]:
neuron = Neuron(2, "RELU")
neuron.predict(data[1][0:2])


[8.913584290546188, 8.063046718945937] [0.7277158617167763, 0.23901038418853748]


6.880706594611355

In [ ]:
def matrix_reshape(matrix):
    cols = len(matrix)
    rows = len(matrix[0])
    elements = (elem for row in matrix for elem in row)
    mT = []
    for i in range(len_x):
        mT.append([])
        for j in range(len_y):
            mT[-1].append(elements.__next__())
    return(mT)